In [1]:
import numpy as np
from pandas import read_csv,to_datetime, DataFrame, pivot, get_dummies

#################
### Load Data ###
#################
Variable_Description = read_csv("./variable_definitions.csv",encoding='latin1')
Dirty_Data = read_csv("./dirty_data.csv",encoding='latin1')
All_colum = Dirty_Data.columns
Cleaned_Data = read_csv("./cleaned_data.csv",encoding='latin1')

######################
### Rename Columns ###
######################
C = Dirty_Data.columns
for c in C:
    if c != 'Rescue_ID':
        Dirty_Data   = Dirty_Data.rename(columns={c: c + '_d'})
        Cleaned_Data = Cleaned_Data.rename(columns={c: c + '_c'})    
print("Number of rows : ",Dirty_Data.shape[0])
display(Dirty_Data.head())
display(Cleaned_Data.head())

###################
### Target Data ###
###################
Order_col = ['Rescue_ID']
for c in C:
    if c != 'Rescue_ID':
        Order_col += [c + '_d', c + '_c']        
Data = Cleaned_Data.merge(Dirty_Data)
Data = Data.fillna("NA")
Data[Order_col].head()

for c in C:
    if c != 'Rescue_ID':
        Data[c + '_Error'] = (Data[c + '_d'] != Data[c + '_c']).map(lambda x : int(x))
        Data.loc[Data[c + '_d'] == 'NA', c + '_d'] = np.nan
        Data.loc[Data[c + '_c'] == 'NA', c + '_c'] = np.nan

############
### Date ###
############
Data['Date_Caught_d'] = Data['Date_Caught_d'].map(lambda x : to_datetime(x,format = "%m/%d/%Y"))
Data['Year'] = Data['Date_Caught_d'].map(lambda x : str(x)[0:4]) 
Data['Month'] = Data['Date_Caught_d'].map(lambda x : str(x)[5:7]) 
Data['Year_Month'] = Data['Date_Caught_d'].map(lambda x : str(x)[0:7])

#########################
### 2010 & 2009  Data ###
#########################
Data = Data.loc[Data.Year.map(lambda x : x in  ('2009','2010')),].reset_index(drop=True).set_index('Rescue_ID')
#Data = Data.set_index('Rescue_ID')
Data.shape

Number of rows :  4292


,Rescue_ID,Date_Caught_d,Researcher_d,CaptureSite_d,ForagingGround_d,CaptureMethod_d,Fisher_d,LandingSite_d,Species_d,ReleaseSite_d,...,CCW_cm_d,Weight_Kg_d,Sex_d,TurtleCharacteristics_d,Status_d,Date_Release_d,Release_Admiss_Notes_d,SpecialRemarks_d,PCVNumber_d,Expenditure_d
0,1998_RE_0001,4/14/1998,researcher_12,site_110,creek,net,fisher_619,site_58,species_1,site_80,...,58.5,NaN,Unknown,NaN,Released,4/14/1998,NaN,NaN,NaN,NaN
1,1998_RE_0002,7/7/1998,researcher_4,not_recorded,ocean,longline,fisher_522,site_22,species_1,not_recorded,...,37.0,NaN,NaN,1B-1LLS,NaN,NaN,NaN,NaN,NaN,NaN
2,1998_RE_0003,8/3/1998,NaN,site_12,creek,net,fisher_1254,not_recorded,species_1,site_109,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998_RE_0004,8/7/1998,researcher_12,site_110,creek,net,fisher_360,not_recorded,species_2,site_113,...,31.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998_RE_0005,9/25/1998,researcher_17,not_recorded,creek,collected floater,fisher_865,site_8,species_3,site_109,...,63.5,NaN,NaN,Bs on C+ old panga wounds,NaN,NaN,Found trapped in mangroves,NaN,NaN,NaN


,Rescue_ID,Date_Caught_c,Researcher_c,CaptureSite_c,ForagingGround_c,CaptureMethod_c,Fisher_c,LandingSite_c,Species_c,ReleaseSite_c,...,CCW_cm_c,Weight_Kg_c,Sex_c,TurtleCharacteristics_c,Status_c,Date_Release_c,Release_Admiss_Notes_c,SpecialRemarks_c,PCVNumber_c,Expenditure_c
0,1998_RE_0001,4/17/1998,researcher_19,site_110,creek,net,fisher_619,site_58,species_1,site_80,...,58.42,NaN,Unknown,NaN,Released,4/17/1998,NaN,NaN,NaN,NaN
1,1998_RE_0002,7/7/1998,researcher_15,site_15,creek,longline,fisher_522,site_22,species_1,not_recorded,...,36.83,NaN,Unknown,1B-1LLS,NaN,NaN,NaN,NaN,NaN,NaN
2,1998_RE_0003,8/3/1998,not_recorded,site_12,creek,not_recorded,fisher_1254,not_recorded,species_1,site_109,...,33.00,NaN,Unknown,NaN,Released,8/3/1998,NaN,NaN,NaN,NaN
3,1998_RE_0004,8/7/1998,researcher_19,site_110,creek,not_recorded,fisher_360,not_recorded,species_2,site_108,...,31.75,NaN,Unknown,There was pillings on carapace.,Released,8/7/1998,NaN,NaN,NaN,NaN
4,1998_RE_0005,9/25/1998,researcher_17,site_8,creek,collected floater,fisher_865,site_8,species_3,site_121,...,63.50,NaN,Unknown,Bs on C+ old panga wounds,Released,9/25/1998,Found trapped in mangroves,NaN,NaN,NaN


(1134, 78)

In [2]:
Test = read_csv("./test_data.csv")
#X_vect = get_dummies(Test[['CaptureSite','LandingSite','ReleaseSite','CaptureMethod','Sex','ForagingGround','Tag_1']].fillna('NA')).columns

### Adding Test Data columns doesn't improse prediction, this is a weird thing

### Build  Model

In [3]:
import pickle
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

Cat_Errors = ['CaptureSite','LandingSite','ReleaseSite',
              'CaptureMethod','Fisher','Tag_1']

Temp = Data[['CaptureSite_d','LandingSite_d','ReleaseSite_d',
             'CaptureMethod_d','Fisher_d','Tag_1_d']].fillna("NA")
for c in Temp.columns:
    Temp = Temp.rename(columns = { c : c[0:-2]})

Train_set = get_dummies(Temp.reset_index() \
                        .merge(Data[[x + '_Error' for x in Cat_Errors]].reset_index()) \
                        .set_index('Rescue_ID'))
X = Train_set[[x for x in Train_set.columns if '_Error' not in x ]]
#for y in [x for x in X_vect if x not in list(X.columns)]:
#    X[y] = 0
y = Train_set[[x + '_Error' for x in Cat_Errors]]

classifier  = GradientBoostingClassifier(init=None,
              learning_rate=1, loss='deviance', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_weight_fraction_leaf=0.0, n_estimators=150,
              presort='auto', random_state=42,verbose=0,
              warm_start=False)
classif_cat = OneVsRestClassifier(classifier)
classif_cat = classif_cat.fit(X, y)

pickle.dump(X.columns,open('./vect_space.p','wb'))
pickle.dump(classif_cat,open('./Cat_model.p','wb'))

### Save Model

In [4]:
svc_classif = pickle.load(open('./Cat_model.p','rb'))
vect_space = pickle.load(open('./vect_space.p','rb'))

Temp = Test[['Rescue_ID'] + Cat_Errors].fillna("NA").set_index('Rescue_ID')

Test_set = get_dummies(Temp)
New_features = [x for x in Test_set if x not in vect_space]
Old_features = [x for x in vect_space if x not in Test_set]
for o in Old_features:
    Test_set[o] = 0


Y_test_pred = DataFrame(svc_classif.predict(Test_set[vect_space]),
                        columns=Cat_Errors,index=Temp.index)


### Number of detections 

In [5]:
Y_test_pred.sum()

CaptureSite       1
LandingSite       8
ReleaseSite      22
CaptureMethod     0
Fisher           57
Tag_1            12
dtype: int64